In [1]:
import cv2
import PIL
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import tensorflow as tf
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt 
import glob, os, re, time, sys, math, ast, json

from PIL import Image
from tqdm import tqdm
from numpy import savetxt
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
#from tensorflow.keras.preprocessing.image import save_img
#from tensorflow.keras.preprocessing.image import img_to_array

import cv2
import PIL
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import tensorflow as tf
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt 
import glob, os, re, time, sys, math, ast, json

from PIL import Image
from tqdm import tqdm
from numpy import savetxt
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img

print('Importing Libraries')

Importing Libraries


In [43]:
def catalog_to_dataframe(line_list):
    res = [ast.literal_eval(x) for x in line_list] 
    dataframe = pd.DataFrame(res)
    dataframe = dataframe.set_index('_index')
    dataframe = dataframe.drop(['_session_id','_timestamp_ms', 'user/mode'], axis = 1)
    return dataframe

def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def make_dataframe(main_dir):
    catalog_files=sorted(glob.glob(main_dir), key=numericalSort)
    myDataFrame = pd.DataFrame()
    for i in range (0, len(catalog_files)):
        infile = open(catalog_files[i]) 
        lines = infile.readlines()
        df = catalog_to_dataframe(lines)
        myDataFrame = myDataFrame.append(df)
    return myDataFrame

def steeing_throttle(n1, n2, n3, n4, s_t, f):
    n=np.concatenate((n1, n2, n3, n4),axis=0)
    nn = n.tolist()
    rounded_nn = [ '%.6f' % i for i in nn ]
    if s_t=='s':
        with open('steering_{}.json'.format(f), 'w', encoding='utf-8') as f:  # saving steering value as json file
            json.dump(nn, f, ensure_ascii=False, indent=4)
    if s_t=='t':
        with open('throttle_{}.json'.format(f), 'w', encoding='utf-8') as f:  # saving throttle value as json file
            json.dump(nn, f, ensure_ascii=False, indent=4)
    else:
        pass
    return rounded_nn

In [45]:
main_dir_1=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/first/*.catalog'
main_dir_2=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/second/*.catalog'
df_1 = make_dataframe(main_dir_1)
df_2 = make_dataframe(main_dir_2)

#Steering column from dataframe
column_s1 = df_1.loc[100:20472,'user/angle']
arr_s1 = column_s1.values
column_s2 = df_2.loc[:,'user/angle']
arr_s2 = column_s2.values

#Throttle column from dataframe
column_t1 = df_1.loc[100:20472,'user/throttle']
arr_t1 = column_t1.values
column_t2 = df_2.loc[:,'user/throttle']
arr_t2 = column_t2.values

dir_new_1=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_1/*.catalog'
dir_new_2=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_2/*.catalog'
df_new_1 = make_dataframe(dir_new_1)
df_new_2 = make_dataframe(dir_new_2)

#Steering column from dataframe
column_new_s1 = df_new_1.loc[217:,'user/angle']
arr_new_s1 = column_new_s1.values
column_new_s2 = df_new_2.loc[:15433,'user/angle']
arr_new_s2 = column_new_s2.values

#Throttle column from dataframe
column_new_t1 = df_new_1.loc[217:,'user/throttle']
arr_new_t1 = column_new_s1.values
column_new_t2 = df_new_2.loc[:15433,'user/throttle']
arr_new_t2 = column_new_t2.values


# steering_list_1=steeing_throttle(arr_s1, arr_s2, s_t='s', f=1) 
# throttle_list_1=steeing_throttle(arr_t1, arr_t2, s_t='t', f=1)

# steering_list_2=steeing_throttle(arr_new_s1, arr_new_s2, s_t='s', f=2) 
# throttle_list_2=steeing_throttle(arr_new_t1, arr_new_t2, s_t='t', f=2)

steering_list_new=steeing_throttle(arr_s1, arr_s2, arr_new_s1, arr_new_s2, s_t='s', f='new') 
throttle_list_new=steeing_throttle(arr_t1, arr_t2, arr_new_t1, arr_new_t2, s_t='t', f='new')

In [46]:
len(arr_s1), len(arr_s2), len(arr_new_s1), len(arr_new_s2), len(steering_list_new), len(throttle_list_new)

(20373, 53426, 40890, 15434, 130123, 130123)

In [5]:
img_dir1="C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/first/*.jpg"
img_dir2="C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/second/*.jpg"
files_1=sorted(glob.glob(img_dir1), key=numericalSort)
files_2=sorted(glob.glob(img_dir2), key=numericalSort)

img_dir_new_1='C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_1/*.jpg'
img_dir_new_2='C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_2/*.jpg'
files_new_1=sorted(glob.glob(img_dir_new_1), key=numericalSort)
files_new_2=sorted(glob.glob(img_dir_new_2), key=numericalSort)

order_new = [files_1[100:20473], files_2, files_new_1[217:], files_new_2[:15434]]
#order_new = [files_new_1, files_new_2]

In [6]:
len(order_new[0]), len(order_new[1]), len(order_new[2]), len(order_new[3])

(20373, 53426, 40890, 15434)

In [7]:
m=0
for i in range(0,len(order_new)):
    for j in order_new[i]:
        # print(j)
        img=cv2.imread(j)
        #cv2.imshow('sample image',img)
        #cv2.waitKey(0) # waits until a key is pressed
        #cv2.destroyAllWindows()
        m+=1
        cv2.imwrite('C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/lane/cv/full/{}.jpg'.format(m), img) #create new folder to save each time to avoid confusion

In [23]:
main_dir_1=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/first/*.catalog'
main_dir_2=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/second/*.catalog'
#main_dir_3=r'C:/Users/Nishanth Siegener/Downloads/Bumblebee/Data/five/*.catalog'
df_1 = make_dataframe(main_dir_1)
df_2 = make_dataframe(main_dir_2)

#Steering column from dataframe
column_s1 = df_1.loc[100:20472,'user/angle']
arr_s1 = column_s1.values

column_s2 = df_2.loc[:,'user/angle']
arr_s2 = column_s2.values

#Throttle column from dataframe
column_t1 = df_1.loc[100:20472,'user/throttle']
arr_t1 = column_t1.values

column_t2 = df_2.loc[:,'user/throttle']
arr_t2 = column_t2.values


#steering_list=steeing_throttle(arr_s1, arr_s2, s_t='s') 
#throttle_list=steeing_throttle(arr_t1, arr_t2, s_t='t')
#r_start=60
#r_end=63
#myDF=myDataFrame.iloc[r_start:-r_end]
#myDF=myDataFrame[0:200]

In [25]:
len(arr_s1)#, len(steering_list)

20373

In [28]:
img_dir1="C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/first/*.jpg"
img_dir2="C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/second/*.jpg"

files_1=sorted(glob.glob(img_dir1), key=numericalSort)
files_2=sorted(glob.glob(img_dir2), key=numericalSort)

row_start=100
row_end=-63
order = [files_1[100:20473], files_2]
# order = files_1[r_start:r_end] + files_2

In [61]:
for i in range(20470,20474):                                  
    img=cv2.imread(files_1[i])
    cv2.imshow('sample image',img)
    cv2.waitKey(0) # waits until a key is pressed
    cv2.destroyAllWindows()

In [50]:
%%time
# Load and save the images using OpenCV

m=0
for i in range(0,len(order)):
    for j in order[i]:
        # print(j)
        img=cv2.imread(j)
        #cv2.imshow('sample image',ima)
        #cv2.waitKey(0) # waits until a key is pressed
        #cv2.destroyAllWindows()
        m+=1
        cv2.imwrite('C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/lane/cv/2/{}.jpg'.format(m), img) # create new folder to save each time to avoid confusion

In [55]:
'''%%time
# Load and save using Tensorflow

imm=[]
n=0
for i in range(0,len(order)):
    for j in order[i]:
        imagess=load_img(j,grayscale=False, color_mode='rgb',target_size= (120,160))
        
        # convert image to a numpy array
        img_array = img_to_array(imagess)
        imm.append(img_array)
        
        # save the image with a new filename
        n+=1
        save_img('C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/lane/tf/2/{}.jpg'.format(n), img_array) # better to create new folder to save each time to avoid confusion
# images_array=np.array(imm)'''
#np.save('images_array', images_array)

"%%time\n# Load and save using Tensorflow\n\nimm=[]\nn=0\nfor i in range(0,len(order)):\n    for j in order[i]:\n        imagess=load_img(j,grayscale=False, color_mode='rgb',target_size= (120,160))\n        \n        # convert image to a numpy array\n        img_array = img_to_array(imagess)\n        imm.append(img_array)\n        \n        # save the image with a new filename\n        n+=1\n        save_img('C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/lane/tf/2/{}.jpg'.format(n), img_array) # better to create new folder to save each time to avoid confusion\n# images_array=np.array(imm)"

In [12]:
def load_images_from_folder(foldername):
    images = []
    for filename in os.listdir(foldername):
        files=sorted(glob.glob(filename), key=numericalSort)
        print(files)
        img = cv2.imread(os.path.join(foldername,filename))
        if img is not None:
            images.append(img)
    return images
img_list=load_images_from_folder("C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/fm/forest")
img_array=np.array(img_list, dtype=float)

[]


TypeError: expected str, bytes or os.PathLike object, not list

In [6]:
len(img_list)

0

In [51]:
foldername="C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/"
im=os.listdir(foldername)
for filename in im:
    print(os.path.join(foldername,filename))

C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/111.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/114.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/115.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/23.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/41.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/46.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/48.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/49.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/8.jpg
C:/Users/Nishanth Siegener/Documents/GitHub/dummy_data_nis/forest/98.jpg


In [43]:
cv2.imshow('sample image',img_array[0])
 
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows()

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel           input_shape=(480, 620, 3) 480/2/2/2/2/2 after max pooling, 620/2/2/2/2/2

        # CNN #input Channel,output channel,size of the kernel,stride,padding
        self.conv1 = nn.Sequential(nn.Conv2d(3, 24, 5, stride=1, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
                                         nn.Dropout2d(p=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(24, 32, 5, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                          nn.Dropout2d(p=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 5, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                         nn.Dropout2d(p=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(64, 64, 3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                          nn.Dropout2d(p=0.2))
        self.conv5 = nn.Sequential(nn.Conv2d(64, 64, 3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                         nn.Dropout2d(p=0.2))

        # pooling
        self.pool = nn.MaxPool2d(2, 2)

        # an affine operation: y = Wx + b
        self.fcx1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        # self.fcx2 = nn.Linear(120, 84)
        # self.fcx3 = nn.Linear(84, 10)

        self.fcy1 = nn.Linear(6, 14)
        self.fcy2 = nn.Linear(14, 14)
        self.fcy3 = nn.Linear(14, 14)

        self.fcz1 = nn.Linear(114, 50)
        self.fcz2 = nn.Linear(50, 20)
        self.fcz3 = nn.Linear(20, 2)

    # noinspection PyUnusedLocal
    def forward(self, x, y):
        x = self.pool(self.first_layer)
        x = self.pool(self.second_layer)
        x = self.pool(self.third_layer)
        x = self.pool(self.fourth_layer)
        x = self.pool(self.fifth_layer)
        # x = x.view(-1, 64 * 15 * 19) #19 or 20 not sure dimension of the image
        x = torch.flatten(x, 1)  # flatten all dimensions except the batch dimension  nn.Dropout(0.2)
        x = F.relu(self.fcx1(x))
        # x = F.relu(self.fcx2(x))
        # x = F.relu(self.fcx3(x))
        y = F.relu(self.fcy1(y))
        y = F.relu(self.fcy2(y))
        y = F.relu(self.fcy3(y))
        z = torch.cat((x, y), dim=1)
        z = F.relu(self.fcz1(y))
        z = F.relu(self.fcz2(y))
        z = F.relu(self.fcz3(y))
        return z


net = Net()
print(net)

Net(
  (first_layer): Sequential(
    (0): Conv2d(3, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.2, inplace=False)
  )
  (second_layer): Sequential(
    (0): Conv2d(24, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.2, inplace=False)
  )
  (third_layer): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.2, inplace=False)
  )
  (fourth_layer): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.

In [12]:
# With Learnable Parameters
m = nn.BatchNorm2d(100)
# Without Learnable Parameters
m = nn.BatchNorm2d(100, affine=False)
input = torch.randn(20, 100, 35, 45)
output = m(input)

In [1]:
output

NameError: name 'output' is not defined

In [ ]:
# Neural network
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
#from keras import Input
from keras.layers import Dense, Dropout, Flatten, Convolution2D
from keras.models import Model
from numpy import concatenate
from tensorflow.keras.backend import concatenate
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


# from torchvision.datasets import MNIST


def conv2d(filters, kernel, strides, layer_num, activation='relu'):
    return Convolution2D(filters=filters,
                         kernel_size=(kernel, kernel),
                         strides=(strides, strides),
                         activation=activation,
                         name='conv2d_' + str(layer_num))


def core_cnn_layers(img_in, drop, l4_stride=1):
    img_in = tf.cast(img_in, dtype=tf.float32)
    x = img_in
    x = conv2d(24, 5, 2, 1)(x)
    x = Dropout(drop)(x)
    x = conv2d(32, 5, 2, 2)(x)
    x = Dropout(drop)(x)
    x = conv2d(64, 5, 2, 3)(x)
    x = Dropout(drop)(x)
    x = conv2d(64, 3, l4_stride, 4)(x)
    x = Dropout(drop)(x)
    x = conv2d(64, 3, 1, 5)(x)
    x = Dropout(drop)(x)
    x = Flatten(name='flattened')(x)
    return x


def default_imu(num_outputs, imu_in, num_imu_inputs, img_in, input_shape):
    drop = 0.2
    im = Input(shape=input_shape, name='img_in')
    imu = Input(shape=num_imu_inputs, name="imu_in")

    x = core_cnn_layers(img_in, drop)
    x = Dense(100, activation='relu')(x)
    x = Dropout(.1)(x)

    y = imu_in
    y = Dense(14, activation='relu')(y)
    y = Dense(14, activation='relu')(y)
    y = Dense(14, activation='relu')(y)

    z = concatenate([x, y])
    z = Dense(50, activation='relu')(z)
    z = Dropout(.1)(z)
    z = Dense(50, activation='relu')(z)
    z = Dropout(.1)(z)
    z = Dense(20, activation='relu')(z)
    z = Dropout(.1)(z)
    outputs = [Dense(1, activation='linear', name='out_' + str(i))(z) for i in range(num_outputs)]

    outputs = tf.convert_to_tensor(outputs, dtype=None, dtype_hint=None, name=None)
    # tf.convert_to_tensor(value, dtype=None, dtype_hint=None, name=None)
    print(outputs)
    model = Model(inputs=[im, imu], outputs=outputs)
    return model


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel           input_shape=(480, 620, 3) 480/2/2/2/2/2 after max pooling, 620/2/2/2/2/2

        # CNN #input Channel,output channel,size of the kernel,padding
        self.first_layer = nn.Sequential(nn.Conv2d(3, 24, 5, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
                                         nn.Dropout2d(p=0.2))
        self.second_layer = nn.Sequential(nn.Conv2d(24, 32, 5, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                          nn.Dropout2d(p=0.2))
        self.third_layer = nn.Sequential(nn.Conv2d(32, 64, 5, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                         nn.Dropout2d(p=0.2))
        self.fourth_layer = nn.Sequential(nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                          nn.Dropout2d(p=0.2))
        self.fifth_layer = nn.Sequential(nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                         nn.Dropout2d(p=0.2))

        # pooling
        self.pool = nn.MaxPool2d(2, 2)

        # an affine operation: y = Wx + b
        self.fcx1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        # self.fcx2 = nn.Linear(120, 84)
        # self.fcx3 = nn.Linear(84, 10)

        self.fcy1 = nn.Linear(6, 14)
        self.fcy2 = nn.Linear(14, 14)
        self.fcy3 = nn.Linear(14, 14)

        self.fcz1 = nn.Linear(114, 50)
        self.fcz2 = nn.Linear(50, 20)
        self.fcz3 = nn.Linear(20, 2)

    # noinspection PyUnusedLocal
    def forward(self, x, y):
        x = self.pool(self.first_layer)
        x = self.pool(self.second_layer)
        x = self.pool(self.third_layer)
        x = self.pool(self.fourth_layer)
        x = self.pool(self.fifth_layer)
        # x = x.view(-1, 64 * 15 * 19) #19 or 20 not sure dimension of the image
        x = torch.flatten(x, 1)  # flatten all dimensions except the batch dimension  nn.Dropout(0.2)
        x = F.relu(self.fcx1(x))
        # x = F.relu(self.fcx2(x))
        # x = F.relu(self.fcx3(x))
        y = F.relu(self.fcy1(y))
        y = F.relu(self.fcy2(y))
        y = F.relu(self.fcy3(y))
        z = torch.cat((x, y), dim=1)
        z = F.relu(self.fcz1(y))
        z = F.relu(self.fcz2(y))
        z = F.relu(self.fcz3(y))
        return z


net = Net()
print(net)
